In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ZDV_finance_events_reasoning_strong_20.json to ZDV_finance_events_reasoning_strong_20.json
User uploaded file "ZDV_finance_events_reasoning_strong_20.json" with length 4760 bytes


In [4]:

import json

with open("ZDV_finance_events_reasoning_strong_20.json", "r", encoding="utf-8") as f:
    zdv_data = json.load(f)

print("Toplam örnek:", len(zdv_data))
print("Örnek veri:", zdv_data[0])


Toplam örnek: 20
Örnek veri: {'query': 'Neden gıda enflasyonu yükseldi?', 'when': 'kuraklık döneminde', 'key': 'sebze ve meyve rekoltesi', 'value': 'ciddi oranda azaldı', 'so': 'arz daraldı ve fiyatlar hızla yükseldi'}


In [5]:

from collections import defaultdict

class Vocab:
    def __init__(self):
        self.token2idx = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2}
        self.idx2token = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>'}
        self.next_index = 3

    def encode(self, text):
        tokens = text.lower().replace('.', '').replace(',', '').split()
        ids = []
        for token in tokens:
            if token not in self.token2idx:
                self.token2idx[token] = self.next_index
                self.idx2token[self.next_index] = token
                self.next_index += 1
            ids.append(self.token2idx[token])
        return [self.token2idx['<SOS>']] + ids + [self.token2idx['<EOS>']]

    def decode(self, ids):
        return ' '.join(self.idx2token.get(i, '?') for i in ids if i > 2)

vocab = Vocab()

q_data, w_data, k_data, v_data, s_data = [], [], [], [], []
for row in zdv_data:
    q_data.append(vocab.encode(row['query']))
    w_data.append(vocab.encode(row['when']))
    k_data.append(vocab.encode(row['key']))
    v_data.append(vocab.encode(row['value']))
    s_data.append(vocab.encode(row['so']))

SOS_IDX = vocab.token2idx['<SOS>']
EOS_IDX = vocab.token2idx['<EOS>']


In [6]:

import torch
from torch.nn.utils.rnn import pad_sequence

def to_tensor(data):
    return pad_sequence([torch.tensor(x) for x in data], batch_first=True)

q_tensor = to_tensor(q_data)
w_tensor = to_tensor(w_data)
k_tensor = to_tensor(k_data)
v_tensor = to_tensor(v_data)
s_tensor = to_tensor(s_data)


In [7]:

import torch.nn as nn

class ECETransformer(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, batch_first=True),
            num_layers
        )
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model, nhead, batch_first=True),
            num_layers
        )
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        memory = self.encoder(src)
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask)
        return self.fc_out(output)


In [8]:

model = ECETransformer(len(vocab.token2idx))
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    model.train()
    total_loss = 0
    for i in range(len(q_tensor)):
        src = torch.cat([q_tensor[i:i+1], w_tensor[i:i+1], k_tensor[i:i+1], v_tensor[i:i+1]], dim=1)
        tgt_input = s_tensor[i:i+1][:,:-1]
        tgt_output = s_tensor[i:i+1][:,1:]
        logits = model(src, tgt_input)
        loss = criterion(logits.view(-1, logits.size(-1)), tgt_output.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(q_tensor):.4f}")


Epoch 1, Loss: 4.7976
Epoch 2, Loss: 3.9988
Epoch 3, Loss: 3.2111
Epoch 4, Loss: 2.5453
Epoch 5, Loss: 2.1460
Epoch 6, Loss: 1.8229
Epoch 7, Loss: 1.5570
Epoch 8, Loss: 1.3467
Epoch 9, Loss: 1.2511
Epoch 10, Loss: 1.1772


In [10]:

import random
from torch.nn.functional import softmax

def sampled_generate(model, q, w, kx, v, max_len=20, min_len=5, temperature=1.0, top_k=5):
    model.eval()
    with torch.no_grad():
        src = torch.cat([q, w, kx, v], dim=1)
        memory = model.encoder(model.embedding(src))
        generated = torch.tensor([[SOS_IDX]], dtype=torch.long, device=src.device)

        for t in range(max_len):
            tgt_emb = model.embedding(generated)
            output = model.decoder(tgt_emb, memory)
            logits = model.fc_out(output[:, -1, :]) / temperature
            logits[:, EOS_IDX] -= 2.0

            probs = softmax(logits, dim=-1)
            topk_probs, topk_indices = torch.topk(probs, top_k)
            next_token = topk_indices[0, torch.multinomial(topk_probs, 1).item()]
            next_token = next_token.view(1, 1)

            generated = torch.cat([generated, next_token], dim=1)

            if next_token.item() == EOS_IDX and t >= min_len:
                break

        return vocab.decode(generated.squeeze().tolist()[1:])

idx = random.randint(0, len(q_tensor) - 1)
output = sampled_generate(model, q_tensor[idx:idx+1], w_tensor[idx:idx+1], k_tensor[idx:idx+1], v_tensor[idx:idx+1])
print("💬 Query:", zdv_data[idx]["query"])
print("🧠 Ground Truth:", zdv_data[idx]["so"])
print("🎲 Model Output:", output)


💬 Query: Neden sağlık harcamaları arttı?
🧠 Ground Truth: sağlık harcamaları yükseldi
🎲 Model Output: lojistik gecikmeleri yaşandı harcamaları geriledi
